In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from PIL import Image
from sklearn.metrics import f1_score
import tensorflow as tf
import datetime

%load_ext autoreload
%autoreload 2
    
import ml_utils.road_seg as rs
import ml_utils.model as model

# 1. Loading and Preprocessing

Load raw images and ground truth

In [ ]:
path_data = 'data'
path_train_dir = os.path.join(path_data, 'training')
path_test = os.path.join(path_data, 'test_set_images')
path_models = 'model'

if not os.path.exists(path_models):
    os.mkdir(path_models)
train_imgs, train_gt, val_imgs, val_gt, id_train, id_valid = rs.load_train_set(path_train_dir, ratio=0.8)

In [ ]:
train_imgs_norm, mean_train, std_train = rs.normalize_data(train_imgs, mode='all')
val_imgs_norm, _, _ = rs.normalize_data(val_imgs, mode='all', mean_ref=mean_train, std_ref=std_train)

Generate smaller images for training

In [ ]:
patch_size = 80
patch_tr, lab_tr,_ = rs.get_patches_all(train_imgs_norm, train_gt, patch_size)
patch_val, lab_val,_ = rs.get_patches_all(val_imgs_norm, val_gt, patch_size)

print('Shapes train: {},  test: {}'.format(patch_tr.shape, patch_val.shape))

Take only part of train and validation set (should at least contain a part of the road)

In [ ]:
useful_patches_tr, useful_lab_tr = rs.get_useful_patches(patch_tr, lab_tr, 0.2, 0.7)
useful_patches_val, useful_lab_val = rs.get_useful_patches(patch_val, lab_val, 0.2, 0.7)
useful_lab_tr = useful_lab_tr.astype(int)
useful_lab_val = useful_lab_val.astype(int)

print('Shapes train: {},  test: {}'.format(useful_patches_tr.shape, useful_patches_val.shape))

# 2. Model training

Display patches example with label ground truth

In [ ]:
n_display = 9
id_display = np.random.permutation(len(useful_patches_tr))[:n_display]

plt.figure(figsize=(16, 5*(n_display//3)))
for i, id_ in enumerate(id_display):
    plt.subplot(n_display//3,3,i+1)
    plt.imshow(useful_patches_tr[id_]); plt.imshow(useful_lab_tr[id_], alpha=0.3); plt.axis('off');

In [ ]:
import ml_utils.model as model

m = model.Model()
m.train_model(useful_patches_tr, useful_lab_tr, train_imgs_norm, train_gt, val_imgs_norm, val_gt, n_epoch=25)
name_model = m.save_path

# 3. Results vizualisation


In [ ]:
m = model.Model(display_log=False)
d = m.get_model_layers(train_imgs[2], 'model/2017_12_03_15h19_model.ckpt')
m.plot_layers(train_imgs_norm[2], d)

In [ ]:
y_pred_train = m.apply_model(train_imgs_norm, 'model/2017_12_03_15h19_model.ckpt')
print('F1-score train: {}'.format(m.predict_f1(train_gt, y_pred_train)))
y_pred_valid = m.apply_model(val_imgs_norm, 'model/2017_12_03_15h19_model.ckpt')
print('F1-score validation: {}'.format(m.predict_f1(val_gt, y_pred_valid)))

In [ ]:
rs.display_predictions(y_pred_train, train_imgs, train_gt)

In [ ]:
rs.display_predictions(y_pred_valid, val_imgs, val_gt)

# 3. Post Process

In [ ]:
test_img = rs.load_test_set(path_test)
test_img_norm, _, _ = rs.normalize_data(test_img, mode='all', mean_ref=mean_train, std_ref=std_train)

In [ ]:
m = model.Model(display_log=False)
y_pred_test = m.apply_model(test_img_norm, 'model/2017_12_03_15h19_model.ckpt')

In [ ]:
im_pred_test = np.reshape(y_pred_test, test_img.shape[:3]).astype(np.float32)
print(test_img.shape)
rs.display_predictions(y_pred_test, test_img)

In [ ]:
import skimage.measure
import skimage.draw
import skimage.morphology

def post_processing(imgs):
    imgs_post = np.zeros(imgs.shape)
    for i in range(imgs.shape[0]):
        imgs_post[i] = skimage.morphology.remove_small_objects(imgs[i].astype(bool), min_size = 200)    
    return imgs_post

im_pred_test_post = post_processing(im_pred_test)

rs.display_predictions(y_pred_test, test_img, im_pred_test_post)

# 4. Submission

In [ ]:
rs.create_submission(im_pred_test_post, 'test_25epoch_classexample_post.csv')